This has been optimized

In [41]:
import time
import os 

start_time = time.time()

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')
if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')

from utils.helpers import divide_chunks

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py


In [42]:
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, List, Optional
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_exponential
from tqdm import tqdm
from functools import lru_cache
import yfinance as yf

In [43]:
# Setup logging
logging.basicConfig(level=logging.INFO)

# %% [Function Definitions]
@lru_cache(maxsize=1000)
def fetch_ticker_info_cached(ticker_symbol: str) -> Dict[str, Optional[float]]:
    """
    Cached retrieval of ticker info from Yahoo Finance to minimize repeated API calls.
    """
    try:
        return yf.Ticker(ticker_symbol).info
    except Exception as e:
        logging.error(f"Error fetching data for {ticker_symbol}: {e}")
        return {}

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=4, max=10)
)

def fetch_with_retry(ticker: str) -> Dict[str, Optional[float]]:
    """
    Retry fetching data for a ticker symbol using tenacity.
    """
    return fetch_ticker_info_cached(ticker)

def get_stock_kpis(ticker: str, kpis: List[str]) -> Dict[str, Optional[float]]:
    """
    Fetch KPIs for a specific stock ticker.
    """
    try:
        info = fetch_with_retry(ticker)
        kpi_data = {kpi: info.get(kpi) for kpi in kpis}
        kpi_data['Tickers'] = ticker  # Add ticker column
        return kpi_data
    except Exception as e:
        logging.error(f"Error processing {ticker}: {e}")
        return {'Tickers': ticker, **{kpi: None for kpi in kpis}}


def batch_process_stocks(tickers: List[str], kpis: List[str], max_workers: int = 10) -> pd.DataFrame:
    """
    Process stock tickers in batches using ThreadPoolExecutor for parallel API calls.
    """
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_ticker = {
            executor.submit(get_stock_kpis, ticker, kpis): ticker
            for ticker in tickers
        }

        for future in tqdm(as_completed(future_to_ticker), total=len(tickers)):
            results.append(future.result())

    # Create DataFrame from results
    df = pd.DataFrame(results)
    return df


In [44]:
# %% [Main Workflow]
# Load the Sharpe ratio DataFrame

sharpe_ratio_df = pd.read_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/sharpe_ratios.csv')
spy_sharpe = sharpe_ratio_df.loc[sharpe_ratio_df['Tickers'] == 'SPY', 'Sharpe_ratios'].values[0]

# Filter only stocks above benchmark
sharpe_ratio_df = sharpe_ratio_df[sharpe_ratio_df['Sharpe_ratios'] >= spy_sharpe]

print(f'Spy Sharpe: {spy_sharpe.round(2)}')

Spy Sharpe: 1.8


In [45]:
num_in_chunks = 50
tickers = sharpe_ratio_df['Tickers'].tolist()

print('\n----Intiating Chunk Process----')
ticker_chunks = list(divide_chunks(tickers, num_in_chunks))
print(f'{len(tickers)} tikcers divided into {len(ticker_chunks)} chunks')
        
essential_kpis = ['marketCap', 'trailingPE', 'profitMargins', 'trailingEps']

df_list = []

# Loop through each chunk and process the stocks
print('\n----Fethcing KPIs----')
for chunk in ticker_chunks:
    kpi_df_chunk = batch_process_stocks(chunk, essential_kpis)
    df_list.append(kpi_df_chunk)

# Concatenate all DataFrames vertically
kpi_df = pd.concat(df_list, axis=0, ignore_index=True)
print(f'KPI Df Shape: {kpi_df.shape}')


----Intiating Chunk Process----
188 tikcers divided into 4 chunks

----Fethcing KPIs----


100%|██████████| 38/38 [00:00<00:00, 71.19it/s]

KPI Df Shape: (188, 5)


In [46]:

print('\n----Combining Sharpe & KPI Df----')
# Combine results with Sharpe ratio data
valuation_df = pd.merge(kpi_df, sharpe_ratio_df, on='Tickers').dropna()
print('\n----Filtering Df----')
# Filter and save
filtered_valuation_df = valuation_df[
    (valuation_df['profitMargins'] > 0.2) &
    (valuation_df['Sharpe_ratios'] > 0.75) &
    (valuation_df['marketCap'] > 10_000_000_000)
].sort_values('profitMargins', ascending=False)

filtered_valuation_df.to_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/filtered_valuation_df.csv', index=False)
logging.info(f"Final filtered DataFrame saved. Rows: {filtered_valuation_df.shape[0]}")

# Print the sorted DataFrame (Optional)

end_time = time.time()
elapsed_time = end_time - start_time
print(f'\nTotal Run Time: {elapsed_time}')
filtered_valuation_df

INFO:root:Final filtered DataFrame saved. Rows: 32



----Combining Sharpe & KPI Df----

----Filtering Df----

Total Run Time: 2.966830015182495


,marketCap,trailingPE,profitMargins,trailingEps,Tickers,Sharpe_ratios
89,3.159026e+10,70.403990,0.65343,19.53,TPL,2.929985
1,3.336518e+12,53.849804,0.55041,2.53,NVDA,2.168000
26,8.650310e+10,8.621622,0.50511,5.92,MO,1.912481
81,2.796168e+10,15.984751,0.41335,12.46,GDDY,2.774124
168,1.103125e+10,26.489052,0.41189,4.11,DTM,4.148844
19,1.788620e+11,8.203279,0.40711,6.10,HSBC,2.013610
100,1.614524e+10,15.889280,0.40305,22.76,UTHR,2.219034
20,1.462581e+11,55.817310,0.40294,2.08,ANET,1.904942
58,5.113407e+10,11.837264,0.39410,4.24,MPLX,2.906674
88,2.651822e+10,8.845455,0.35139,7.70,SYF,2.775834
